<a href="https://colab.research.google.com/github/atick-faisal/Crowd-Emotion/blob/main/src_v4/CE_TE_MobileNet_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import joblib
import numpy as np
import tensorflow as tf

from sklearn.utils import shuffle
from sklearn.metrics import classification_report, accuracy_score

tf.__version__

'2.4.1'

In [2]:
BASE_DIR         = '/content/drive/MyDrive/Research/Crowd Emotion Val/'
DATASET_DIR      = 'Dataset_AF/'
FOLDS            = ['Fold 1', 'Fold 2', 'Fold 3', 'Fold 4', 'Fold 5']
EMOTIONS         = ['Approval', 'Disapproval', 'Neutral']

IMG_LEN          = 160
IMG_SIZE         = (IMG_LEN, IMG_LEN)
DATASET_LEN      = 14882
DATASET_LEN_VAL  = 9487

CONFIG           = 'LOG_SPEC_W400_H200_HAMM_IMG' + str(IMG_LEN)

TEST_FOLD        = 'Fold 1'
EPOCHS           = 10
LEARNING_RATE    = 0.001

In [3]:
path = os.path.join(BASE_DIR, DATASET_DIR)
X    = joblib.load(path + CONFIG + '_X.joblib')
y    = joblib.load(path + CONFIG + '_y.joblib')
f    = joblib.load(path + CONFIG + '_f.joblib')

In [4]:
# mask      = (f == FOLDS.index(TEST_FOLD))
# X_train   = X[~mask, :]
# X_test    = X[mask, :]
# y_train   = y[~mask, :]
# y_test    = y[mask, :]

In [5]:
X, y = shuffle(X, y)
# X_train     = X[:12000, :]
# y_train     = y[:12000, :]
# X_test      = X[12000:, :]
# y_test      = y[12000:, :]

In [6]:
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input
rescale = tf.keras.layers.experimental.preprocessing.Rescaling(1./127.5, offset= -1)

In [7]:
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')
base_model.trainable = False

9412608/9406464 [==============================] - 0s 0us/step


In [8]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(len(EMOTIONS))

In [9]:
def get_model():
    inputs = tf.keras.Input(shape=IMG_SHAPE)
    x = preprocess_input(inputs)
    x = base_model(x, training=False)
    x = global_average_layer(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    outputs = prediction_layer(x)
    model = tf.keras.Model(inputs, outputs)
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=LEARNING_RATE),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
    return model

In [10]:
model = get_model()
# X_train, y_train = shuffle(X_train, y_train)
history = model.fit(X, y, epochs=EPOCHS)
model.save(os.path.join(BASE_DIR, 'TE_MOBILENET_ALL_DATA'))

Epoch 1/10
297/297 [==============================] - 41s 22ms/step - loss: 0.3461 - accuracy: 0.8825
Epoch 2/10
297/297 [==============================] - 6s 21ms/step - loss: 0.0614 - accuracy: 0.9852
Epoch 3/10
297/297 [==============================] - 6s 21ms/step - loss: 0.0392 - accuracy: 0.9902
Epoch 4/10
297/297 [==============================] - 6s 21ms/step - loss: 0.0264 - accuracy: 0.9931
Epoch 5/10
297/297 [==============================] - 6s 21ms/step - loss: 0.0239 - accuracy: 0.9943
Epoch 6/10
297/297 [==============================] - 6s 21ms/step - loss: 0.0204 - accuracy: 0.9948
Epoch 7/10
297/297 [==============================] - 6s 22ms/step - loss: 0.0175 - accuracy: 0.9945
Epoch 8/10
297/297 [==============================] - 6s 22ms/step - loss: 0.0188 - accuracy: 0.9955
Epoch 9/10
297/297 [==============================] - 6s 22ms/step - loss: 0.0130 - accuracy: 0.9960
Epoch 10/10
297/297 [==============================] - 6s 22ms/step - loss: 0.0118 - accur

In [11]:
# prob = tf.keras.Sequential([model, tf.keras.layers.Softmax()])
# y_pred_hot = prob.predict(X_test)
# y_pred = np.argmax(y_pred_hot, axis=1)
# print(classification_report(y_test.ravel(), y_pred))